In [1]:
# Import the Python 3 print function
from __future__ import print_function
import pandas as pd
import numpy as np
from osgeo import gdal
from osgeo import gdal_array
import os


In [2]:
np.set_printoptions(threshold=np. inf)
np.seterr(invalid='ignore') # ignore dividing by zero or infinity
B_RED = 2 # red band number
B_NIR = 3 # nir band number


In [3]:
folder_path = 'C:/Users/sci/Desktop/images' # main folder where data stored
crops = os.listdir(folder_path) # get crops names within the folder
barley = crops[0] # assign names of crops as a variable
soya = crops[1]
wheat = crops[2]

In [4]:
def create_field_images(folder_path:str,crop_name:str,field)-> []:
    """Создает список изобрежении,то есть tiff файлов из каталога поле
        Вход: корневой каталог,имя культуры и имя поле. 
        Выход: список снимоков в указонном катологе т.е для 
        указанного поля
    """
    
    return  [f for f in os.listdir(folder_path +'/' + crop_name + '/' + field) if f.endswith('.tif')]# creating list of tif images 

In [5]:
def calculate_ndvi_for_image(dataset):
    """
    Загружает изоброжение по указаноому пути и возращает посчитанный NDVI
    Вход: Open a GDAL dataset (gdal.Open)
    Выход: массив ndvi
    
    """
    image_datatype = dataset.GetRasterBand(1).DataType
    image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount),
                 dtype=gdal_array.GDALTypeCodeToNumericTypeCode(image_datatype))
    for b in range(dataset.RasterCount): # Loop over all bands in dataset
        band = dataset.GetRasterBand(b + 1) # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
        image[:, :, b] = band.ReadAsArray() # Read in the band's data into the third dimension of our array
        
        ndvi = (image[:, :, B_NIR] - image[:, :, B_RED]) / \
                (image[:, :, B_NIR] + image[:, :, B_RED]).astype(np.float64)

        C=(ndvi[~np.isnan(ndvi)])
        
    return C

In [18]:
# Open a GDAL dataset
def calculate_ndvi_for_crop(crop_name:str):
    fields = os.listdir(folder_path +'/' + crop_name )
    field_images = []
    df = pd.DataFrame(columns = ['field','max_ndvi','mean_ndvi','median_ndvi','min_ndvi','st_dev_ndvi','date','image'])
    for field in fields:
        field_images = create_field_images(folder_path, crop_name, field)
        for field_image in field_images:
            dataset = gdal.Open(folder_path +'/' + crop_name + '/' + field + '/' + field_image, gdal.GA_ReadOnly)
            C = calculate_ndvi_for_image(dataset)
            print(f'appending ndvi values for {field_image} image.Crop {crop_name}.field--{field}')
            new_row = {'field':field_image[11:-4],'max_ndvi':C.max(),'mean_ndvi':C.mean()
                            ,'median_ndvi':np.median(C),'min_ndvi':C.min(),'st_dev_ndvi':C.std(),
                            'date':field_image[:10],'image':field_image}
            df= df.append(new_row, ignore_index=True) 
    return  df

In [19]:
def export_ndvi_values(crops: []):
    """Записавает значения ndvi в одтельные exel файлы по каждой культуре
        вход: список культур
    """
    for crop_name in crops:
        excelname = f'{crop_name}_ndvi_values.xlsx'
        crop_df = calculate_ndvi_for_crop(crop_name)
        print(f'Saving excel file with ndvi values for crop {crop_name}')
        crop_df.to_excel(excelname)
    return print(f'Exel Files Saved in {os.getcwd()}')

In [20]:
export_ndvi_values(crops)

appending ndvi values for 07-05-2020_2-3.tif image.Crop barley.field--2_3_114,7
appending ndvi values for 09-05-2020_2-3.tif image.Crop barley.field--2_3_114,7
appending ndvi values for 11-09-2020_2-3.tif image.Crop barley.field--2_3_114,7
appending ndvi values for 14-05-2020_2-3.tif image.Crop barley.field--2_3_114,7
appending ndvi values for 16-07-2020_2-3.tif image.Crop barley.field--2_3_114,7
appending ndvi values for 23-06-2020_2-3.tif image.Crop barley.field--2_3_114,7
appending ndvi values for 26-06-2020_2-3.tif image.Crop barley.field--2_3_114,7
appending ndvi values for 29-05-2020_2-3.tif image.Crop barley.field--2_3_114,7
appending ndvi values for 07-05-2020_4-3-2.tif image.Crop barley.field--4_3_2_63.3
appending ndvi values for 09-05-2020_4-3-2.tif image.Crop barley.field--4_3_2_63.3
appending ndvi values for 11-09-2020_4-3-2.tif image.Crop barley.field--4_3_2_63.3
appending ndvi values for 14-05-2020_4-3-2.tif image.Crop barley.field--4_3_2_63.3
appending ndvi values for 16